In [2]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm 
import intake
import fsspec
import seaborn as sns
import math
import cartopy.crs as ccrs
import cartopy.feature as cf
import os
import cdsapi

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

<ipython-input-2-fcc32bc5f501>:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
def get_lat_name(ds):
    for lat_name in ['latitude','lat']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ds.dims) - {'time'}
    return (ds * weight).mean(other_dims)

def alaska_mean(ds):
    lat_slice = ds.sel(lat=slice(60,71))
    ak_slice = lat_slice.sel(lon=slice(193,219))
    lat = ak_slice[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ak_slice.dims) - {'time'}
    return (ak_slice * weight).mean(other_dims)

def arctic_mean(ds):
    lat_slice = ds.sel(latitude=slice(90,60))
    lat = lat_slice[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(lat_slice.dims) - {'time'}
    return (lat_slice * weight).mean(other_dims)

In [ ]:
def load_all_models(model_name):
    tas_anomaly_hist = xr.open_dataset('/nobackup/users/lulur/tas-anomaly_'+model_name+'_historical.nc')
    tas_anomaly_ssp126 = xr.open_dataset('/nobackup/users/lulur/tas-anomaly_'+model_name+'_ssp126.nc')
    tas_anomaly_ssp245 = xr.open_dataset('/nobackup/users/lulur/tas-anomaly_'+model_name+'_ssp245.nc')
    tas_anomaly_ssp370 = xr.open_dataset('/nobackup/users/lulur/tas-anomaly_'+model_name+'_ssp370.nc')
    tas_anomaly_ssp585 = xr.open_dataset('/nobackup/users/lulur/tas-anomaly_'+model_name+'_ssp585.nc')
    tas_anomalies = [tas_anomaly_hist,tas_anomaly_ssp126,tas_anomaly_ssp245,tas_anomaly_ssp370,tas_anomaly_ssp585]
    return tas_anomalies

In [ ]:
BCC_tas_anomalies = load_all_models('BCC-CSM2-MR')
#WACCM_tas_anomalies = load_all_models('CESM2-WACCM')
ACCESS_CM2_tas_anomalies = load_all_models('ACCESS-CM2') #this one works but doesn't have ssp370

In [ ]:
def calc_ann_max(msts):
    years = []
    year_maxes = []
    for yr, tas in msts.groupby('time.year'):
        years.append(yr)
        max_temp = -100
        for t in tas['tas']:
            #print(tas)
            if t > max_temp:
                max_temp = t
        year_maxes.append(max_temp)
    
    return years, year_maxes

In [5]:
def download_day_mx2t(year,month,day):
    """
    year, month and day are all strings giving the day being requested. If month or day are single-digit, 
    must be in the format '01','02', etc.
    """
    #variable shortname is mx2t
    c = cdsapi.Client()
    r = c.retrieve(
        'reanalysis-era5-single-levels', {
                'variable'    : 'maximum_2m_temperature_since_previous_post_processing',
                'product_type': 'reanalysis',
                'year'        : year,
                'month'       : month,
                'day'         : day,
                'time'        : [
                    '00:00','01:00','02:00',
                    '03:00','04:00','05:00',
                    '06:00','07:00','08:00',
                    '09:00','10:00','11:00',
                    '12:00','13:00','14:00',
                    '15:00','16:00','17:00',
                    '18:00','19:00','20:00',
                    '21:00','22:00','23:00'
                ],
                'format'      : 'netcdf'
        })
    r.download('/nobackup/users/lulur/ERA5_day_temps/mx2t_'+year+month+day+'.nc')

def delete_day_mx2t(year,month,day):
    os.remove('/nobackup/users/lulur/ERA5_day_temps/mx2t_'+year+month+day+'.nc')

def find_arctic_mean_daily_max(year,month,day):
    download_day_mx2t(year,month,day)
    ds = xr.open_dataset('/nobackup/users/lulur/ERA5_day_temps/mx2t_'+year+month+day+'.nc')
    mean_temp = arctic_mean(ds)
    dailymax = mean_temp.max(dim='time')
    delete_day_mx2t(year,month,day)
    
    return float(dailymax['mx2t'])

def find_arctic_daily_maxes(year):
    months = ['01','02','03','04','05','06','07','08','09','10','11','12']
    days31 = ['01','03','05','07','08','10','12']
    daily_maxes = []
    for month in months:
        if month in days31:
            days = np.arange(1,32)
        elif month == '02':
            days = np.arange(1,29)
        else:
            days = np.arange(1,31)
        for day in days:
            if len(str(day))==1:
                day = '0'+str(day)
            else:
                day = str(day)
            print('finding max of month '+month+', day '+day)
            dailymax = find_arctic_mean_daily_max(year, month, day)
            daily_maxes.append(dailymax)
    
    return daily_maxes

def find_arctic_mean_daily_mean(year,month,day):
    download_day_mx2t(year,month,day)
    ds = xr.open_dataset('/nobackup/users/lulur/ERA5_day_temps/mx2t_'+year+month+day+'.nc')
    mean_temp = arctic_mean(ds)
    dailymax = mean_temp.mean(dim='time')
    delete_day_mx2t(year,month,day)
    
    return float(dailymax['mx2t'])

def find_arctic_summer_daily_means(year):
    months = ['05','06','07','08','09']
    days31 = ['01','03','05','07','08','10','12']
    daily_means = []
    for month in months:
        if month in days31:
            days = np.arange(1,32)
        elif month == '02':
            days = np.arange(1,29)
        else:
            days = np.arange(1,31)
        for day in days:
            if len(str(day))==1:
                day = '0'+str(day)
            else:
                day = str(day)
            print('finding mean of month '+month+', day '+day)
            dailymean = find_arctic_mean_daily_mean(year, month, day)
            daily_means.append(dailymean)
    
    return daily_means

#use xarray.ufuncs.maximum

In [10]:
np.savetxt('/nobackup/users/lulur/ERA5_day_temps/mx2t_means_2016',means2016)

In [11]:
means2014 = find_arctic_summer_daily_means('2014')
np.savetxt('/nobackup/users/lulur/ERA5_day_temps/mx2t_means_2016',means2014)
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)


finding mean of month 05, day 01


2021-07-16 18:23:31,706 INFO Welcome to the CDS
2021-07-16 18:23:31,707 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-07-16 18:23:31,805 INFO Request is queued
2021-07-16 18:23:32,887 INFO Request is running
2021-07-16 18:23:40,264 INFO Request is completed
2021-07-16 18:23:40,265 INFO Downloading https://download-0003.copernicus-climate.eu/cache-compute-0003/cache/data8/adaptor.mars.internal-1626474216.2339873-18364-14-4420f6f8-f74a-4d55-8a42-7d4611137374.nc to /nobackup/users/lulur/ERA5_day_temps/mx2t_20140501.nc (47.5M)
2021-07-16 18:23:58,875 INFO Download rate 2.6M/s   
2021-07-16 18:23:59,111 INFO Welcome to the CDS
2021-07-16 18:23:59,112 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-07-16 18:23:59,200 INFO Request is queued


finding mean of month 05, day 02


2021-07-16 18:24:00,283 INFO Request is running
2021-07-16 18:24:07,663 INFO Request is completed
2021-07-16 18:24:07,663 INFO Downloading https://download-0000.copernicus-climate.eu/cache-compute-0000/cache/data2/adaptor.mars.internal-1626474243.493895-12317-15-ac3f5c8d-41b4-4680-b88d-e9c579611be7.nc to /nobackup/users/lulur/ERA5_day_temps/mx2t_20140502.nc (47.5M)
2021-07-16 18:24:12,573 INFO Download rate 9.7M/s   
2021-07-16 18:24:12,798 INFO Welcome to the CDS
2021-07-16 18:24:12,798 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-07-16 18:24:12,886 INFO Request is queued


finding mean of month 05, day 03


...

finding mean of month 09, day 29


2021-07-16 19:26:15,074 INFO Request is running
2021-07-16 19:26:22,449 INFO Request is completed
2021-07-16 19:26:22,449 INFO Downloading https://download-0000.copernicus-climate.eu/cache-compute-0000/cache/data2/adaptor.mars.internal-1626477978.34709-11410-3-598f4682-9d14-4575-90bb-ac672a7a03a5.nc to /nobackup/users/lulur/ERA5_day_temps/mx2t_20140929.nc (47.5M)
2021-07-16 19:27:08,551 INFO Download rate 1M/s     
2021-07-16 19:27:08,776 INFO Welcome to the CDS
2021-07-16 19:27:08,777 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-07-16 19:27:08,865 INFO Request is queued


finding mean of month 09, day 30


2021-07-16 19:27:09,949 INFO Request is running
2021-07-16 19:27:17,892 INFO Request is completed
2021-07-16 19:27:17,893 INFO Downloading https://download-0009.copernicus-climate.eu/cache-compute-0009/cache/data3/adaptor.mars.internal-1626478033.0881424-21904-4-ee0e4d42-73e4-49c9-978a-d939781ab43c.nc to /nobackup/users/lulur/ERA5_day_temps/mx2t_20140930.nc (47.5M)
2021-07-16 19:27:21,224 INFO Download rate 14.3M/s  


NameError: name 'datetime' is not defined

In [9]:
len(means2016)

153

In [ ]:
max2016 = find_arctic_daily_maxes('2016')
np.savetxt('/nobackup/users/lulur/ERA5_day_temps/mx2t_2016',max2016)
#dailymax = find_arctic_mean_daily_max('2016','01','01')
print(max2016)
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)


In [ ]:
max2016

In [ ]:
print(float(dailymax['mx2t']))

In [ ]:

#variabel shortname is mx2t
c = cdsapi.Client()
r = c.retrieve(
    'reanalysis-era5-single-levels', {
            'variable'    : 'maximum_2m_temperature_since_previous_post_processing',
            'product_type': 'reanalysis',
            'year'        : '2017',
            'month'       : '01',
            'day'         : ['01', '02'],
            'time'        : [
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ],
            'format'      : 'netcdf'
    })
r.download('mx2t_20170101-20170102.nc')


In [ ]:
import time, sys
from datetime import datetime, timedelta
 
from netCDF4 import Dataset, date2num, num2date

In [ ]:
day = 20170101
d = datetime.strptime(str(day), '%Y%m%d')
f_in = 'mx2t_%d-%s.nc' % (day, (d + timedelta(days = 1)).strftime('%Y%m%d'))
f_out = 'daily-mx2t_%d.nc' % day
dataset = xr.open_dataset(f_in)
mean_temp = arctic_mean(dataset)
#dailymax = dataset.max(dim='time', skipna=None)

In [ ]:
dailymax = mean_temp.max(dim='time')
dailymax

In [ ]:
day = 20170101
d = datetime.strptime(str(day), '%Y%m%d')
f_in = 'mx2t_%d-%s.nc' % (day, (d + timedelta(days = 1)).strftime('%Y%m%d'))
f_out = 'daily-mx2t_%d.nc' % day
 
time_needed = []
for i in range(1, 25):
    time_needed.append(d + timedelta(hours = i))

with Dataset(f_in) as ds_src:
    var_time = ds_src.variables['time']
    time_avail = num2date(var_time[:], var_time.units,
            calendar = var_time.calendar)
 
    indices = []
    for tm in time_needed:
        a = np.where(time_avail == tm)[0]
        if len(a) == 0:
            sys.stderr.write('Error: precipitation data is missing/incomplete - %s!\n'
                    % tm.strftime('%Y%m%d %H:%M:%S'))
            sys.exit(200)
        else:
            #print('Found %s' % tm.strftime('%Y%m%d %H:%M:%S'))
            indices.append(a[0])
 
    var_mx2t = ds_src.variables['mx2t']
    max_daily_temp = -1000
    for idx in indices:
        data = var_mx2t[idx, :, :]
        print(data.shape)
        if data > max_daily_temp:
            max_daily_temp = data
            
#     with Dataset(f_out, mode = 'w', format = 'NETCDF3_64BIT_OFFSET') as ds_dest:
#         # Dimensions
#         for name in ['latitude', 'longitude']:
#             dim_src = ds_src.dimensions[name]
#             ds_dest.createDimension(name, dim_src.size)
#             var_src = ds_src.variables[name]
#             var_dest = ds_dest.createVariable(name, var_src.datatype, (name,))
#             var_dest[:] = var_src[:]
#             var_dest.setncattr('units', var_src.units)
#             var_dest.setncattr('long_name', var_src.long_name)
 
#         ds_dest.createDimension('time', None)
#         var = ds_dest.createVariable('time', np.int32, ('time',))
#         time_units = 'hours since 1900-01-01 00:00:00'
#         time_cal = 'gregorian'
#         var[:] = date2num([d], units = time_units, calendar = time_cal)
#         var.setncattr('units', time_units)
#         var.setncattr('long_name', 'time')
#         var.setncattr('calendar', time_cal)
 
#         # Variables
#         var = ds_dest.createVariable(var_tp.name, np.double, var_tp.dimensions)
#         var[0, :, :] = data
#         var.setncattr('units', var_tp.units)
#         var.setncattr('long_name', var_tp.long_name)
 
#         # Attributes
#         ds_dest.setncattr('Conventions', 'CF-1.6')
#         ds_dest.setncattr('history', '%s %s'
#                 % (datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
#                 ' '.join(time.tzname)))
 
#         print('Done! Daily total precipitation saved in %s' % f_out)

In [ ]:
dailymax

In [ ]:
maximum = dailymax.max(('latitude'))
maximum

In [ ]:
plt.contourf([dailymax['latitude'],dailymax['longitude'],],dailymax['mx2t'])